In [21]:
import numpy as np
import pandas as pd
import requests


In [22]:
df = pd.read_excel('c:/temp/naolang_logs.xlsx', sheet_name='Programs')
filtered = df.loc[1:,['When Added', 'Program']]

In [23]:
filtered

,When Added,Program
1,2020-11-26 21:03:16.820,reset()\nstart{\n[_~e?$-;Gk)bst{#xdfsP]positio...
2,2020-11-26 21:04:03.958,reset()\nstart{\n[_~e?$-;Gk)bst{#xdfsP]positio...
3,2020-11-26 21:05:00.142,reset()\nstart{\n[pD-T!_.mOgyA#OL?98Re]positio...
4,2020-11-26 21:08:19.118,reset()\nstart{\n[9o?^DD89QGcImPD7fCg5]changeL...
5,2020-11-26 21:08:34.273,reset()\nstart{\n[9o?^DD89QGcImPD7fCg5]changeL...
...,...,...
137,2020-11-27 01:57:40.697,reset()\nstart{\n[44EKq5-c8#R#+C8/33}R]changeL...
138,2020-11-27 01:57:56.730,reset()\nstart{\n[44EKq5-c8#R#+C8/33}R]changeL...
139,2020-11-27 01:58:44.775,reset()\nstart{\n[44EKq5-c8#R#+C8/33}R]changeL...
140,2020-11-27 01:59:39.883,reset()\nstart{\n[44EKq5-c8#R#+C8/33}R]changeL...


In [45]:
headers = {'Content-Type': 'application/json; charset=utf-8'}
data = {'name': 'bob','password': 'bob'}
r = requests.post('http://localhost:5000/api/v1/session', json = data, headers = headers)
r.raise_for_status()
token = r.json()['output']['token']
headers['Authorization'] = 'Bearer %s' % (token, )

In [46]:
data = {'code': 'reset()'}
r = requests.post('http://localhost:5000/api/v1/code/compile', json = data, headers = headers)
r.raise_for_status()
r.json()

{'output': {'nodes': [{'token': {'lineNumber': 0,
     'linePosition': 0,
     'type': 'Identifier',
     'value': 'reset'},
    'type': 'Function'}]},
 'successful': True}

In [130]:
def compile(code):
    data = {'code': code}
    r = requests.post('http://localhost:5000/api/v1/code/compile', json = data, headers = headers)
    r.raise_for_status()
    return r.json()['output']['nodes']

filtered['Compiled'] = filtered.apply(lambda row: compile(row['Program']), axis=1)

In [139]:
def formatToken(token):
    value = token['value']
    if token['type'] == 'Text':
        return "'%s'" % (value, )
    return value

def formatNode(node):
    args = ''
    if 'arguments' in node:
        args = '(' + ','.join(map(formatNode, node['arguments'])) +  ')'
    children = ''
    if 'children' in node:
        children = '->[' + ','.join(map(formatNode, node['children'])) +  ']'
    return node['type'] + ':' + formatToken(node['token']) + args + children

def format(nodes):
    start_node = nodes[1]
    if 'children' in start_node:
        return '\n'.join(map(formatNode, start_node['children']))
    else:
        return ''

filtered['Output'] = filtered.apply(lambda row: format(row['Compiled']), axis=1)

In [140]:
filtered.loc[10,'Output']

"Function:say(Constant:'Ssssssstttttttttooooooooopppppppppp iiiiiittttt')"

In [141]:
filtered.to_excel('c:/temp/formatted.xlsx', sheet_name='logs')